In [3]:
import os
import pickle
import sqlite3
import pandas as pd

In [ ]:
# Connect to database
conn = sqlite3.connect("fake_news.db")
cursor = conn.cursor()

# Load TF-IDF features
cursor.execute("SELECT data FROM features WHERE type = 'tfidf'")
X_tfidf_compressed = cursor.fetchone()[0]
X_tfidf = pickle.loads(X_tfidf_compressed)

conn.close()

print("✅ Features loaded successfully!")

In [ ]:
conn = sqlite3